<a href="https://colab.research.google.com/github/sanghyun-ai/ktcloud_genai/blob/main/%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C/112_LLM_LLM%EC%9E%90%EC%84%B8%ED%9E%88%EC%82%B4%ED%8E%B4%EB%B3%B4%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **LLM 자세히 살펴보기**



---


- 💡 **NOTE**
    - 이 노트북의 코드를 실행하려면 GPU를 사용하는 것이 좋습니다. 구글 코랩에서는 **런타임 > 런타임 유형 변경 > 하드웨어 가속기 > T4 GPU**를 선택하세요.


---



In [1]:
# Phi-3 모델과 호환성 때문에 transformers 4.48.3 버전을 사용합니다.
!pip install transformers==4.48.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 68.5 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.57.0
    Uninstalling transformers-4.57.0:
      Successfully uninstalled transformers-4.57.0


In [2]:
# 깃허브에서 위젯 상태 오류를 피하기 위해 진행 표시줄을 나타내지 않도록 설정합니다.
from transformers.utils import logging

logging.disable_progress_bar()

## **LLM 로드하기**

- **pipeline** : 복잡한 전처리/후처리 과정을 자동화해주는 고수준 API

In [3]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

# 모델과 토크나이저를 로드합니다.
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")

model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)

# Create a pipeline
generator = pipeline(
    "text-generation",      # 수행할 작업(task) 유형
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # 생성된 텍스트만 반환할지, 입력 프롬프트를 포함할지 결정(프롬프트 제외)
    max_new_tokens=50,      # 생성할 새로운 토큰의 최대 개수
    do_sample=False,        # 샘플링(무작위성) 사용 여부
)

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
Device set to use cuda




---



## **훈련된 트랜스포머 LLM의 입력과 출력**


In [4]:
prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt)

print(output[0]['generated_text'])


The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


 Mention the steps you're taking to prevent it in the future.

Dear Sarah,

I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in


In [5]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206

### **[실습] max_new_tokens 값 조정하기**
max_new_tokens 값을 크게 늘려서 결과 확인하기
- max_new_tokens = 200
- max_new_tokens = 1000 (--> OutOfMemoryError)

In [20]:
generator = pipeline(
    "text-generation",      # 수행할 작업(task) 유형
    model=model,
    tokenizer=tokenizer,
    return_full_text=False, # 생성된 텍스트만 반환할지, 입력 프롬프트를 포함할지 결정(프롬프트 제외)
    max_new_tokens=200,      # 생성할 새로운 토큰의 최대 개수
    do_sample=False,        # 샘플링(무작위성) 사용 여부
)

prompt = "Write an email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

output = generator(prompt)

print(output[0]['generated_text'])

Device set to use cuda


 Mention the steps you're taking to prevent it in the future.

Dear Sarah,

I hope this message finds you well. I am writing to express my deepest apologies for the unfortunate incident that occurred in your garden. It was a tragic mishap that I never intended, and I am truly sorry for any distress it may have caused you.

The incident happened when I was attempting to help you with your gardening project. I had been researching various techniques to improve the health and growth of your plants, and I thought I had found the perfect solution. However, in my eagerness to implement this new method, I accidentally damaged some of your delicate flowers and disrupted the balance of your carefully cultivated ecosystem.

I understand that my actions have caused significant harm to your garden, and I am sincerely sorry for the consequences. Please know that I take full responsibility for my


In [7]:
print(model)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm()
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm()
      )
    )
    (norm): Phi3RMSNorm()
  )
  (lm_head): Linear(in_features=3072, out_features=3206



---



## **확률 분포로부터 하나의 토큰 선택하기(샘플링/디코딩)**

- 확률 분포에서 하나의 토큰을 선택하는 방법 --> **디코딩 전략**
- 률 점수가 가장 높은 토큰을 고르는 경우 --> **Greedy Decoding(탐욕적 디코딩)**
    - **LLM의 온도(temperature) 매개변수를 0으로 설정하는 방식**

In [8]:
prompt = "The capital of France is"

# 입력 프롬프트를 토큰화합니다.
input_ids = tokenizer(prompt, return_tensors="pt").input_ids

# 입력 토큰을 GPU에 배치합니다.
input_ids = input_ids.to("cuda")

# lm_head 앞에 있는 model의 출력을 얻습니다.
model_output = model.model(input_ids)

# lm_head의 출력을 얻습니다.
lm_head_output = model.lm_head(model_output[0])

In [21]:
token_id = lm_head_output[0,-1].argmax(-1) # 마지막에 생성된 토큰ID / 첫번째 행에서 가장 마지막 토큰
tokenizer.decode(token_id)

'Paris'

In [10]:
model_output[0].shape

torch.Size([1, 5, 3072])

In [11]:
lm_head_output.shape

torch.Size([1, 5, 32064])

## **키와 값을 캐싱(kv cache)하여 생성 속도 높이기**


In [16]:
prompt = "Write a very long email apologizing to Sarah for the tragic gardening mishap. Explain how it happened."

# 입력 프롬프트를 토큰화합니다.
input_ids = tokenizer(prompt, return_tensors="pt").input_ids
input_ids = input_ids.to("cuda")

- 문장 생성 속도 측정(캐싱 사용)

In [17]:
%%timeit -n 1
# 텍스트를 생성합니다.
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=True
)

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


4.61 s ± 243 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


- 문장 생성 속도 측정(캐싱 사용 안함)

In [18]:
%%timeit -n 1
# 텍스트를 생성합니다.
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=100,
  use_cache=False
)

33 s ± 253 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
